Gian Luca Malvezzi Petini

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Input, LeakyReLU, Conv2D, Conv2DTranspose, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from keras import Model

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/GANS/CelebA", label_mode=None, image_size=(32, 32), batch_size=64
)
dataset = dataset.map(lambda x: x / 255.0)

Found 30000 files belonging to 1 classes.


In [ ]:
discriminator = tf.keras.Sequential(
    [
        Input(shape=(32, 32, 3)),
        Conv2D(64, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Flatten(),
        Dropout(0.2),
        Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 16, 16, 64)        3136      
                                                                 
 leaky_re_lu_44 (LeakyReLU)  (None, 16, 16, 64)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 8, 8, 128)         131200    
                                                                 
 leaky_re_lu_45 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 4, 128)         262272    
                                                                 
 leaky_re_lu_46 (LeakyReLU)  (None, 4, 4, 128)         0         
                                                                 
 flatten_3 (Flatten)         (None, 2048)            

In [ ]:
latent_dim = 64

generator = tf.keras.Sequential(
    [
        Input(shape=(latent_dim,)),
        Dense(8 * 8 * 128),
        Reshape((8, 8, 128)),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8192)              532480    
                                                                 
 reshape_9 (Reshape)         (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_27 (Conv2D  (None, 16, 16, 128)      262272    
 Transpose)                                                      
                                                                 
 leaky_re_lu_42 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_28 (Conv2D  (None, 32, 32, 256)      524544    
 Transpose)                                                      
                                                                 
 leaky_re_lu_43 (LeakyReLU)  (None, 32, 32, 256)       0 

In [ ]:
class GAN(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        generated_images = self.generator(random_latent_vectors)

        combined_images = tf.concat([generated_images, real_images], axis=0)

        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        misleading_labels = tf.zeros((batch_size, 1))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))

In [ ]:
epochs = 50

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=Adam(learning_rate=0.0001),
    g_optimizer=Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

Epoch 1/50
469/469 [==============================] - 6638s 14s/step - d_loss: 0.4215 - g_loss: 1.5968
Epoch 2/50
469/469 [==============================] - 38s 81ms/step - d_loss: 0.5494 - g_loss: 1.4334
Epoch 3/50
469/469 [==============================] - 39s 81ms/step - d_loss: 0.5556 - g_loss: 1.2595
Epoch 4/50
469/469 [==============================] - 39s 81ms/step - d_loss: 0.5969 - g_loss: 1.1220
Epoch 5/50
469/469 [==============================] - 39s 81ms/step - d_loss: 0.5528 - g_loss: 1.5548
Epoch 6/50
469/469 [==============================] - 40s 84ms/step - d_loss: 0.5824 - g_loss: 1.1160
Epoch 7/50
469/469 [==============================] - 39s 81ms/step - d_loss: 0.6039 - g_loss: 1.0826
Epoch 8/50
469/469 [==============================] - 40s 84ms/step - d_loss: 0.6113 - g_loss: 1.1563
Epoch 9/50
469/469 [==============================] - 40s 84ms/step - d_loss: 0.5960 - g_loss: 1.0748
Epoch 10/50
469/469 [==============================] - 39s 83ms/step - d_loss: 0.